In [57]:
# Import Module
import pyodbc as dbc
import pandas as pd
 
# Testing
#O_hostname = "win8039.site4now.net"
#O_username = "rah143-001"
#O_password = "sccgroup6"

#Keystone
K_hostname = "ftp.ekeystone.com"
K_username = "s151667"
K_password = "wia16mct"
K_port = 990

#CWR
S_hostname = "ftp.ekeystone.com"
S_username = "S4004302"
S_password = "wii43vcc"
S_port = 990

#Meyer
M_hostname = "ftp.werrv.com"
M_username = "SCCGroup6@ftp.werrv.com"
M_password = "WeeRv666"
M_port = 21

Suppliers = ["Keystone", "Seawide", "Meyer"]

In [58]:
import ftplib
import ssl

class ImplicitFTP_TLS(ftplib.FTP_TLS):
    """
    FTP_TLS subclass that automatically wraps sockets in SSL to support implicit FTPS.
    Prefer explicit TLS whenever possible.
    """

    def __init__(self, *args, **kwargs):
        """Initialise self."""
        super().__init__(*args, **kwargs)
        self._sock = None

    @property
    def sock(self):
        """Return the socket."""
        return self._sock

    @sock.setter
    def sock(self, value):
        """When modifying the socket, ensure that it is SSL wrapped."""
        if value is not None and not isinstance(value, ssl.SSLSocket):
            value = self.context.wrap_socket(value)
        self._sock = value

    def ntransfercmd(self, cmd, rest=None):
        """Override the ntransfercmd method"""
        conn, size = ftplib.FTP.ntransfercmd(self, cmd, rest)
        conn = self.sock.context.wrap_socket(
            conn, server_hostname=self.host, session=self.sock.session
        )
        return conn, size
    
def ftp_connection(hostname,username,password,port,filename):
    
    # create an FTP client instance, use the timeout(seconds) parameter for slow connections only
    ftp = ImplicitFTP_TLS()
    
    # connect to the FTP server
    ftp.connect(host=hostname, port=port)
    
    # login to the FTP server
    ftp.login(user=username, passwd=password)
    
    # setup secure data connection
    ftp.prot_p()
    
    # force UTF-8 encoding
    ftp.encoding = "utf-8"

    # Write file in binary mode
    with open(filename, "wb") as file:
        # Command for Downloading the file "RETR filename"
        ftp.retrbinary(f"RETR {filename}", file.write)

    from io import BytesIO
    fp = BytesIO()
    ftp.retrbinary('RETR ' + filename, fp.write, 1024)

    # reset the cursor to the start of the file stream
    fp.seek(0)
    return pd.read_csv(fp)

In [59]:
for S in Suppliers:
    try:
     if S == "Keystone":
      df_Keystone = ftp_connection(K_hostname, K_username, K_password,K_port,"Inventory.csv")
     elif S == "CWR":
      df_Seawide = ftp_connection(S_hostname, S_username, S_password,S_port,"Inventory.csv")     
     elif S == "Meyer":
        filename = "Meyer Inventory.csv"
        #df_Meyer = ftp_connection(M_hostname, M_username, M_password,M_port,'"Meyer Inventory.csv"') 
        ftp_server = ftplib.FTP(M_hostname, M_username, M_password)
        # Write file in binary mode
        with open(filename, "wb") as file:
          # Command for Downloading the file "RETR filename"
          ftp_server.retrbinary(f"RETR {filename}", file.write)
        from io import BytesIO
        fp = BytesIO()
        ftp_server.retrbinary('RETR ' + filename, fp.write, 1024)

        # reset the cursor to the start of the file stream
        fp.seek(0)
        df_Meyer = pd.read_csv(fp)
    #else:
     #df_CWR = ftp_connection(C_hostname, C_username, C_password,"")
    except Exception as e: print(e)

In [60]:
df_Keystone.head()

,VendorName,VCPN,VendorCode,PartNumber,NTPProductNumber,ManufacturerPartNo,LongDescription,JobberPrice,Cost,Purolatorable,...,AAIACode,IsHazmat,IsChemical,Puralator_Ground_Assessorials,EastQty,PacificNWQty,GreatLakesQty,TotalQty,KitComponents,IsKit
0,AIR LIFT,A1310530,A13,"=""10530""","=""""","=""10530""",HOSE CUTTER,32.14,32.14,True,...,BBCG,False,False,0.0,0,0,2,2,NaN,False
1,AIR LIFT,A1316060,A13,"=""16060""","=""15-0600""","=""16060""",COMP VIARIR BLK BOX,121.53,121.53,True,...,BBCG,False,False,0.0,15,10,21,46,NaN,False
2,AIR LIFT,A1316130,A13,"=""16130""","=""""","=""16130""",COMPRESSOR 12V,243.34,243.34,True,...,BBCG,False,False,0.0,3,2,1,6,NaN,False
3,AIR LIFT,A1316190,A13,"=""16190""","=""""","=""16190""",COMPRSR 12V 1.94 CFM,379.97,379.97,True,...,BBCG,False,False,0.0,2,0,0,2,NaN,False
4,AIR LIFT,A1316192,A13,"=""16192""","=""""","=""16192""",AIR COMPRESSOR FOR 25980EZ HELPER S,183.96,183.96,True,...,BBCG,False,False,0.0,17,0,0,17,NaN,False


In [61]:
df_Seawide.head()

,VendorName,VCPN,VendorCode,PartNumber,LongDescription,AAIACode,ManufacturerPartNo,SeaWideProductNumber,Enhanced_Description,Media_URL,...,UPCCode,IsHazmat,IsChemical,Puralator_Ground_Assessorials,EastQty,PacificNWQty,GreatLakesQty,TotalQty,KitComponents,IsKit
0,AIR LIFT,A1310530,A13,"=""10530""",HOSE CUTTER,BBCG,"=""10530""",NaN,Use to Cut Air Lift Air Line,https://vehiclepartimages.com/ImageServer/Imag...,...,"=""729199105302""",False,False,0.0,0,0,2,2,NaN,False
1,AIR LIFT,A1316060,A13,"=""16060""",COMP VIARIR BLK BOX,BBCG,"=""16060""",NaN,Stationary; 100 PSI; 0.79 Cubic Feet Per Minut...,https://vehiclepartimages.com/ImageServer/Imag...,...,"=""729199160608""",False,False,0.0,15,10,21,46,NaN,False
2,AIR LIFT,A1316130,A13,"=""16130""",COMPRESSOR 12V,BBCG,"=""16130""",NaN,Stationary; 130 PSI; 0.6 Cubic Feet Per Minute...,https://vehiclepartimages.com/ImageServer/Imag...,...,"=""729199161308""",False,False,0.0,3,2,1,6,NaN,False
3,AIR LIFT,A1316190,A13,"=""16190""",COMPRSR 12V 1.94 CFM,BBCG,"=""16190""",NaN,Stationary; 150 PSI; 0.81 Cubic Feet Per Minut...,https://vehiclepartimages.com/ImageServer/Imag...,...,"=""729199161902""",False,False,0.0,2,0,0,2,NaN,False
4,AIR LIFT,A1316192,A13,"=""16192""",AIR COMPRESSOR FOR 25980EZ HELPER S,BBCG,"=""16192""",NaN,12 Volts; With Wiring/ Hose; Replacement For 2...,https://vehiclepartimages.com/ImageServer/Imag...,...,"=""729199160929""",False,False,0.0,17,0,0,17,NaN,False


In [62]:
df_Meyer.head()

,MFGName,MFG Item Number,Item Number,Available,LTL,MFG Qty Available,Stocking,Special Order,Oversize,Addtl Handling Charge
0,3-D Mats,1781-A,DDD1781-A,0,False,NaN,No,Yes,No,No
1,3-D Mats,1781-B,DDD1781-B,0,False,NaN,No,Yes,No,No
2,3-D Mats,1781-C,DDD1781-C,0,False,NaN,No,Yes,No,No
3,3-D Mats,1781-D,DDD1781-D,0,False,NaN,No,Yes,No,No
4,3-D Mats,1781-E,DDD1781-E,0,False,NaN,No,Yes,No,No


In [64]:
import urllib
from sqlalchemy import create_engine
from sqlalchemy import event
import time

driver = '{SQL Server Native Client 11.0}'
server = 'sql8004.site4now.net' # to specify an alternate port
database = 'db_a94243_werrv' 
username = 'db_a94243_werrv_admin' 
password = 'WerRv666'

params = 'DRIVER='+driver + ';SERVER='+server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password
db_params = urllib.parse.quote_plus(params)

engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))


@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(
       conn, cursor, statement, params, context, executemany
        ):
            if executemany:
                cursor.fast_executemany = True
                
try:
  current_time = time.strftime("%H:%M:%S")
  print("The current time is", current_time)
    
  df_Keystone.to_sql("Keystone_Inventory_test", engine, index=False, if_exists="append", schema="dbo")
  df_Meyer.to_sql("Meyer_Inventory_test", engine, index=False, if_exists="append", schema="dbo")

  current_time = time.strftime("%H:%M:%S")
  print("The current time is", current_time)
except Exception as e: print(e)

The current time is 21:27:48


<ipython-input-64-dee4e1765aee>:29: UserWarning: The provided table name 'Keystone_Inventory_test' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_Keystone.to_sql("Keystone_Inventory_test", engine, index=False, if_exists="append", schema="dbo")


The current time is 22:00:47


In [86]:
#pip install flask-sqlalchemy --user

In [84]:
#pip show sqlalchemy

In [93]:
#pip install pymssql

In [96]:
df = pd.read_csv("Inventory.csv",dtype = {'PartNumber': object})

In [95]:
#print(df.dtypes)   

In [97]:
df.tail()

,VendorName,VCPN,VendorCode,PartNumber,NTPProductNumber,ManufacturerPartNo,LongDescription,JobberPrice,Cost,Purolatorable,...,AAIACode,IsHazmat,IsChemical,Puralator_Ground_Assessorials,EastQty,PacificNWQty,GreatLakesQty,TotalQty,KitComponents,IsKit
122596,ZONE OFFROAD,ZORZONT2601,ZOR,"=""ZONT2601""","=""""","=""ZONT2601""",05-15 TACOMA BOX KIT 2 OF 3,854.63,854.63,True,...,FQMS,False,False,49.92,1,1,0,2,NaN,False
122597,ZONE OFFROAD,ZORZONT2602,ZOR,"=""ZONT2602""","=""""","=""ZONT2602""",05-15 TACOMA BOX KIT 3 OF 3,261.73,261.73,True,...,FQMS,False,False,0.00,2,1,0,3,NaN,False
122598,ZONE OFFROAD,ZORZONT2612,ZOR,"=""ZONT2612""","=""""","=""ZONT2612""",2016 TACOMA BOX KIT 3 OF 3,261.73,261.73,True,...,FQMS,False,False,0.00,2,2,0,4,NaN,False
122599,ZONE OFFROAD,ZORZONT3N,ZOR,"=""ZONT3N""","=""""","=""ZONT3N""",LIFT KIT SUSPENSION,1996.96,1996.96,False,...,FQMS,False,False,64.60,1,1,0,2,ZORZON7953-2|ZORZONT2409-1|ZORZONT2600-1|ZORZO...,True
122600,ZONE OFFROAD,ZORZONT4N,ZOR,"=""ZONT4N""","=""""","=""ZONT4N""",LIFT KIT SUSPENSION,1912.26,1912.26,False,...,FQMS,False,False,64.60,2,1,0,3,ZORZON7954-2|ZORZONT2309-1|ZORZONT2401-1|ZORZO...,True
